This part of the code collects a dictionary of all lables and their descriptions

In [3]:
import os
import json
import pandas as pd


def extract_us_gaap_elements():
    data = []
    current_directory = './qtrly-data/'#os.getcwd()

    # Loop through all files in the current directory
    for filename in os.listdir(current_directory):
        if filename.endswith('.json'):
            file_path = os.path.join(current_directory, filename)

            # Open and parse the JSON file
            with open(file_path, 'r') as file:
                try:
                    json_data = json.load(file)
                    us_gaap_elements = json_data.get(
                        'facts', {}).get('us-gaap', {})

                    # Extract relevant fields
                    for metric, details in us_gaap_elements.items():
                        label = details.get('label', '')
                        description = details.get('description', '')
                        data.append(
                            {'metrics': metric, 'label': label, 'description': description})
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON in file {filename}: {e}")

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Remove duplicates
    # df = df.drop_duplicates()

    return df


# Example usage
df = extract_us_gaap_elements()
print(df)

                                                  metrics  \
0                                  AccountsPayableCurrent   
1                            AccountsReceivableNetCurrent   
2                AccrualForEnvironmentalLossContingencies   
3       AccrualForEnvironmentalLossContingenciesDiscou...   
4           AccrualForEnvironmentalLossContingenciesGross   
...                                                   ...   
275278             RestructuringReserveAccrualAdjustment1   
275279  AssetsOfDisposalGroupIncludingDiscontinuedOper...   
275280  DisposalGroupIncludingDiscontinuedOperationCon...   
275281                      FinanceLeasePrincipalPayments   
275282  LiabilitiesOfDisposalGroupIncludingDiscontinue...   

                                                    label  \
0                               Accounts Payable, Current   
1       Accounts Receivable, after Allowance for Credi...   
2            Accrual for Environmental Loss Contingencies   
3       Accrual for Env

In [4]:
df.to_csv('us_gaap_elements.csv', index=False)

In [2]:
import os 
os.listdir('./qtrly-data/')

['VICI.json',
 'T.json',
 'PHM.json',
 'ADM.json',
 'PAYX.json',
 'APD.json',
 'ANET.json',
 'SBUX.json',
 'JKHY.json',
 'ITW.json',
 'OKE.json',
 'MRNA.json',
 'IFF.json',
 'PPG.json',
 'BEN.json',
 'PKG.json',
 'GE.json',
 'SYF.json',
 'WBA.json',
 'CBRE.json',
 'WM.json',
 'HOLX.json',
 'A.json',
 'EMN.json',
 'BMY.json',
 'TSCO.json',
 'AXON.json',
 'CTAS.json',
 'NCLH.json',
 'PSX.json',
 'LMT.json',
 'SNPS.json',
 'PSA.json',
 'AMP.json',
 'TXN.json',
 'HST.json',
 'ETN.json',
 'CZR.json',
 'USB.json',
 'NTRS.json',
 'APH.json',
 'ROK.json',
 'MAS.json',
 'CSGP.json',
 'RF.json',
 'FIS.json',
 'MDT.json',
 'HIG.json',
 'TPL.json',
 'CAG.json',
 'XOM.json',
 'NWS.json',
 'STZ.json',
 'PANW.json',
 'AME.json',
 'WDAY.json',
 'GPN.json',
 'BAC.json',
 'CEG.json',
 'CI.json',
 'AMD.json',
 'NTAP.json',
 'L.json',
 'WELL.json',
 'AFL.json',
 'ECL.json',
 'OTIS.json',
 'SYK.json',
 'AES.json',
 'ARE.json',
 'MCK.json',
 'MAR.json',
 'ZBH.json',
 'ALB.json',
 'HUM.json',
 'ED.json',
 'J

In [7]:
metrics_df = pd.read_csv('us_gaap_elements.csv')
metric_counts = metrics_df['metrics'].value_counts()


print(metric_counts)

metrics
LiabilitiesAndStockholdersEquity                                       488
NetCashProvidedByUsedInInvestingActivities                             488
NetCashProvidedByUsedInFinancingActivities                             488
Assets                                                                 488
NetCashProvidedByUsedInOperatingActivities                             485
                                                                      ... 
DebtSecuritiesRealizedGainLossExcludingOtherThanTemporaryImpairment      1
ContractReceivableRetainageDueInNextRollingTwelveMonths                  1
BillingsInExcessOfCostNoncurrent                                         1
OtherRevenueExpenseFromRealEstateOperations                              1
SubsequentEventEffectOfChangeInTaxStatus                                 1
Name: count, Length: 8336, dtype: int64


Items to collect 
income statement
    Ops
        AccumulatedOtherComprehensiveIncomeLossNetOfTax
    WeightedAverageNumberOfDilutedSharesOutstanding	
    EarningsPerShareDiluted	
Balance Sheet
    Assets
    LiabilitiesAndStockholdersEquity 
Cashflow statement
    NetCashProvidedByUsedInFinancingActivities       
    NetCashProvidedByUsedInInvestingActivities   
    NetCashProvidedByUsedInOperatingActivities




Code to extract diluted EPS from all files. 

In [18]:
import os
import json
import pandas as pd


def extract_metrics(metrics_and_units):
    """
    Extracts specified metrics and their units from JSON files in the current directory.

    Args:
        metrics_and_units: A dictionary where keys are metric names 
                           and values are their corresponding units.

    Returns:
        A pandas DataFrame containing the extracted data.
    """
    data = []
    current_directory = os.getcwd()

    for filename in os.listdir(current_directory):
        if filename.endswith('.json'):
            file_path = os.path.join(current_directory, filename)
            ticker = os.path.splitext(filename)[0]

            try:
                with open(file_path, 'r') as file:
                    json_data = json.load(file)
                    us_gaap_elements = json_data.get(
                        'facts', {}).get('us-gaap', {})

                    for metric, unit in metrics_and_units.items():
                        metric_data = us_gaap_elements.get(
                            metric, {}).get('units', {})
                        if unit in metric_data:
                            for entry in metric_data[unit]:
                                entry_data = {
                                    'ticker': ticker, 'metric': metric}
                                entry_data.update(entry)
                                data.append(entry_data)

            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in file {filename}: {e}")

    return pd.DataFrame(data)


# Define metrics and their units
metrics_and_units = {
    "EarningsPerShareDiluted": "USD/shares",
    "IncomeLossFromContinuingOperationsPerDilutedShare": "USD/shares",
    "IncomeLossFromDiscontinuedOperationsNetOfTaxPerDilutedShare": "USD/shares",
    "WeightedAverageNumberOfDilutedSharesOutstanding": "shares",
    "NetIncomeLoss": "USD",
    "NetIncomeLossFromContinuingOperationsAvailableToCommonShareholdersDiluted": "USD",
    "NetIncomeLossFromDiscontinuedOperationsAvailableToCommonShareholdersDiluted": "USD"

}

# Extract data using the defined metrics
df = extract_metrics(metrics_and_units)
df.to_csv('financial_metrics_extract.csv', index=False)

In [19]:
df = pd.read_csv('financial_metrics_extract.csv')
#this code makes the metrics into columns. the filed sorting is to remove duplicates more than anything
pivot_df = df.sort_values(by='filed', ascending=False).pivot_table(index=['ticker', 'start', 'end',
                                                                          'form', 'filed'], columns='metric', values='val', aggfunc='first').reset_index()
#print(pivot_df.head())

# to do list
# build logic from the parameters by ETL
# get a robust data set where there is only one entry per period


In [22]:
#This code will group by ticker, start, and end, and get only one valid row per period (the latest non-null row)
# Function to get the latest non-null row
def get_latest_non_null_row(group):
    return group.dropna(subset=['filed']).sort_values(by='filed', ascending=False).iloc[0]

# Group by ticker, start, and end, then apply the function
group_df = pivot_df.groupby(['ticker', 'start', 'end'],group_keys=False).apply(get_latest_non_null_row).reset_index(drop=True)

#print(group_df.head())

/var/folders/zl/jy5jnx1j7w12x8wpwfsn9n6m0000gq/T/ipykernel_2510/2134367526.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group_df = pivot_df.groupby(['ticker', 'start', 'end'],group_keys=False).apply(get_latest_non_null_row).reset_index(drop=True)


In [23]:
#This code aims to eliminiate missing EPS values
# Function to calculate the sum while ignoring NaN values
def fill_eps(row):
    if pd.isna(row['EarningsPerShareDiluted']):
        return row[['IncomeLossFromContinuingOperationsPerDilutedShare', 'IncomeLossFromDiscontinuedOperationsNetOfTaxPerDilutedShare']].sum(skipna=True)
    return row['EarningsPerShareDiluted']

# Apply the function to fill missing EPSdiluted values
group_df['EarningsPerShareDiluted'] = group_df.apply(fill_eps, axis=1)
try:
    group_df.drop(columns=['IncomeLossFromContinuingOperationsPerDilutedShare', 'IncomeLossFromDiscontinuedOperationsNetOfTaxPerDilutedShare'], inplace=True)
except KeyError:
    print("Columns already dropped")
eps_df = group_df[['ticker', 'start', 'end','form','filed', 'EarningsPerShareDiluted']]
eps_df.to_csv('eps_data.csv', index=False)